<a href="https://colab.research.google.com/github/GraLesl/MAT-421/blob/main/ModuleC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Root Finding

Often times, it is critical to find the roots of a given function. For common and simple functions, a formula is occasionally availible which can be used to directly calculate the exact roots of a funciton. However, for more complicated functions, this is rarley the case and as such other methods must be employed.

## Problem Statement

Using the integrated fsolve function from scipy to solve for the root of $f(x) = cos(x) - x$ near $-2$.

In [2]:
import numpy as np
from scipy import optimize

f = lambda x: np.cos(x) - x
r = optimize.fsolve(f,-2)
print("r =", r)

result = f(r)
print("result =", result)

r = [0.73908513]
result = [0.]


One interesting idea is what would happen when a given function has no roots such as $f(x) = 1/x$. We will use the *full output* of fsolve to gain more insight.

In [3]:
f = lambda x: 1/x

r, infodict, ier, mesg = optimize.fsolve(f, -2, full_output=True)
print("r =", r)

result = f(r)
print("result =", result)

print(mesg)

r = [-3.52047359e+83]
result = [-2.84052692e-84]
The number of calls to function has reached maxfev = 400.


Here, the solution was not found due to the program reaching an error after not finding a proper root afer a given amount of iterations. One way this can be sidestepped is through the concept of tolerance.

## Tolerance

Tolerance is defined as the level of acceptable error for a given application. As such, we can consider values within tolerance to the true solution as correct when it comes to engineering applications.

This idea can be applied in a few different ways to the root solving problem. One way seeks to find a solution which the $|f(x)|$ is a measure of the error, so the smaller the number the liklier we are to be at a root. Another approach, assuming the solver is iterative and the some value $x_i$ is the *i*th guess of an algorithm, then the change in guesses or $|x_{i+1} - x_i|$ is measured as the erorr. Both these processes have their pros and cons and can be used on an application by application basis.

## Bisection Method

One aproach to an algorithmic root finding utilizes the intermediate value theorum. Assuming a continuous $f(x)$ where the $sign(f(a)) =/= sign(f(b))$, then there must be some point c where $a<c<b$ and $f(c) = 0$.


Using this concept, with a given initial guess at a and b where a and b are real scalars, $ a < b$ and $f(a) > 0 , f(b) < 0$, then a value $m = (b+a)/2$ is the midpoint between a and b. If $f(m)$ is within tolerance of zero, then m is a root. Otherwise, if $f(m) < 0$, m is an improvement on the lower bound and becomes the new b value. Vice versa when $f(m) > 0$. This process is programed below.

In [4]:
import numpy as np
def my_bisection(f,a,b,tol):

  if np.sign(f(a)) == np.sign(f(b)):
    raise Exception("The scalars a and b do not bound a root")

  m = (a + b) / 2

  if np.abs(f(m)) < tol:
    return m
  elif np.sign(f(a)) == np.sign(f(m)):
    return my_bisection(f,m,b,tol)
  elif np.sign(f(b)) == np.sign(f(m)):
    return my_bisection(f,a,m,tol)


f = lambda x: x**2 - 2

r1 = my_bisection(f, 0,2,0.1)
print("r1 =", r1)
r01 = my_bisection(f, 0,2,0.01)
print("r01 =", r01)

print("f(r1) =", f(r1))
print("f(r01) =", f(r01))

r1 = 1.4375
r01 = 1.4140625
f(r1) = 0.06640625
f(r01) = -0.00042724609375


## Newton-Rhapson Method

Another option for root finding is the Newton-Rhapson method which takes $f(x)$ to be a smooth and continuous function, an initial guess $x_0$, and some unkonwn root $x_r$. Assuming $x_0 =/= x_r$, this approach makes successive iterations stepping closer and closer to the final root.

Using the formula $x_i = x_{i-1} - \frac{g(x_{i-1})}{g'(x_{i-1})}$. A next guess can be made which steps closer to the functions root. This process can be repeated until a final value is achieved.


In [5]:
from re import X
import numpy as np

def my_newton(f,df,x0,tol):

  if abs(f(x0)) < tol:
    return x0
  else:
    return my_newton(f,df,x0 - f(x0)/df(x0), tol)

f = lambda x: x**2 - 2
f_prime = lambda x: 2*X

estimate = my_newton(f, f_prime, 1.5, 1e-6)
print("estimate =", estimate)
print("sqrt(2) =", np.sqrt(2))

estimate = 1.4142139123093318
sqrt(2) = 1.4142135623730951


As can be seen, both the np.sqrt function and the function $f(x)$ approahc a very similar value when solving for the square root of 2.

That being said, this solution can have its downfalls. Take for example the polynomial $ f(x) = x^3 - 100x^2 - x + 100$. This polynomial has roots at $x = 1$ and $x = 100$. If an initial guess is made at $x_0 = 0$ an interesting thing happens.

Using the forumla from above the next step guess becomes $x1 = 0 - \frac{100}{-1} = 100$. While this is a root of the initial function, it is much further from the initial guess of 0 which is very close to 1.